# Correlation Analysis

In [17]:
import os
import json
import pandas as pd
import numpy as np
import scipy
from scipy.stats import wilcoxon
from scipy.spatial import distance
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.float_format = '{:,.3f}'.format


In [18]:
shorten_error_model_name = {
    "random" : "RND",
    "error_model_triphone_rich" : "TR",
    "error_model_pure_diversity" : "PD",
    "error_model_without_diversity_enhancing" : "IC-WDE",
    "error_model" : "IC",
    "asrevolve_error_model_real" : "ASR-EV",
    "word_error_predictor_real/no_word_enhance" : "NWE",
    "word_error_predictor_real/word_enhance": "WE"
}

shorten_finetuned_model_name = {
    "random": "RND",
    "triphone_rich": "TR",
    "pure_diversity": "PD",
    "icassp_without_diversity_enhancing_real_mix": "IC-WDE",
    "icassp_real_mix": "IC",
    "asrevolve_error_model_real": "ASR-EV",
    "word_error_real_mix/no_word_enhance": "NWE",
    "word_error_real_mix/word_enhance": "WE"
}

def shorten_em_name(tools):
    return [shorten_error_model_name[tool] for tool in tools]

def shorten_ft_name(tools) :
    return [shorten_finetuned_model_name[tool] for tool in tools]

tool_short_names = ["RND", "TR", "PD", "IC-WDE", "IC", "ASR-EV", "NWE", "WE"]

finetuned_model_tool_names = ["random", "triphone_rich", "pure_diversity", "icassp_without_diversity_enhancing_real_mix", "icassp_real_mix",
         "asrevolve_error_model_real", "word_error_real_mix/no_word_enhance", "word_error_real_mix/word_enhance"]





### The relative improvement of WER after fine-tuning

In [3]:
with open('result/RQ2.json', 'r') as f:
  data = json.load(f)

asrs = ["quartznet", "hubert", "wav2vec-base"]
datasets = ["YBAA", "ZHAA", "ASI", "TNI", "NCC",
            "TXHC", "EBVS", "ERMS", "YDCK", "YKWK", "THV", "TLV"]
tools = ["random", "triphone_rich", "pure_diversity", "icassp_without_diversity_enhancing_real_mix", "icassp_real_mix",
         "asrevolve_error_model_real", "word_error_real_mix/no_word_enhance", "word_error_real_mix/word_enhance"]

finetuned_model_performance_on_test_set = {}
for asr in asrs:
    finetuned_model_performance_on_test_set[asr] = {}
    for dataset in datasets:
        finetuned_model_performance_on_test_set[asr][dataset] = {}
        for tool in tools:
            finetuned_model_performance_on_test_set[asr][dataset][shorten_finetuned_model_name[tool]] = pd.read_csv(
                data[asr][dataset][tool])

# finetuned_model_performance_on_test_set


In [4]:
with open('result/original.json', 'r') as f:
  original_data = json.load(f)

original_model_performance_on_test_set = {}

for asr in asrs:
    original_model_performance_on_test_set[asr] = {}
    for dataset in datasets:
        original_model_performance_on_test_set[asr][dataset] = original_data[asr][dataset]["test"]["wer"]

# original_model_performance_on_test_set



In [5]:
relative_improvement_of_finetuned_model = {}
for asr in asrs:
    relative_improvement_of_finetuned_model[asr] = {}
    for dataset in datasets:
        relative_improvement_of_finetuned_model[asr][dataset] = {}
        for tool in shorten_ft_name(tools):
            relative_improvement_of_finetuned_model[asr][dataset][tool] = {}
            for metric in ["WER_Seed1", "WER_Seed2", "WER_Seed3", "WER_Avg"]:
                relative_improvement_of_finetuned_model[asr][dataset][tool][metric] = (
                    (original_model_performance_on_test_set[asr][dataset] - finetuned_model_performance_on_test_set[asr][dataset][tool][metric]) / original_model_performance_on_test_set[asr][dataset]).to_list()

# relative_improvement_of_finetuned_model


### 1. Correlation between the original model's WER on the selected samples and the relative improvement of WER after fine-tuning

In [6]:
with open('result/RQ1.json', 'r') as f:
  rq1_data = json.load(f)


original_model_performance_on_selected_samples = {}

asrs = ["quartznet", "hubert", "wav2vec-base"]
datasets = ["YBAA", "ZHAA", "ASI", "TNI", "NCC",
            "TXHC", "EBVS", "ERMS", "YDCK", "YKWK", "THV", "TLV"]
tools = ["random", "error_model_triphone_rich", "error_model_pure_diversity", "error_model_without_diversity_enhancing", "error_model",
         "asrevolve_error_model_real", "word_error_predictor_real/no_word_enhance", "word_error_predictor_real/word_enhance"]

original_model_performance_on_selected_samples = {}
for asr in asrs:
    original_model_performance_on_selected_samples[asr] = {}
    for dataset in datasets:
        original_model_performance_on_selected_samples[asr][dataset] = {}
        for tool in tools:
            original_model_performance_on_selected_samples[asr][dataset][shorten_error_model_name[tool]] = {}
            for metric in ["WER_Seed1", "WER_Seed2", "WER_Seed3", "WER_Avg"]:
                original_model_performance_on_selected_samples[asr][dataset][shorten_error_model_name[tool]][metric] = pd.read_csv(
                  rq1_data[asr][dataset][tool])[metric].tolist()

# original_model_performance_on_selected_samples


In [7]:
arr_1 = []
arr_2 = []

for asr in asrs:
    for dataset in datasets:
        for tool in tool_short_names:
            for metric in ["WER_Seed1", "WER_Seed2", "WER_Seed3"]:
                arr_1 += original_model_performance_on_selected_samples[asr][dataset][tool][metric]
                arr_2 += relative_improvement_of_finetuned_model[asr][dataset][tool][metric]

assert len(arr_1) == len(arr_2)
            

In [8]:
scipy.stats.pearsonr(arr_1, arr_2)

(-0.004170908880369349, 0.8063706949422409)

### 3. Correlation between "the distance of triphone rich distribution and ideal distribution on the selected samples" and the relative improvement of WER after fine-tuning

In [10]:
with open('result/triphone_rich.json', 'r') as f:
  triphone_rich_data = json.load(f)


In [30]:
## without any grouping mechanism

arr_1 = []
arr_2 = []

for asr in asrs:
    for dataset in datasets:
        tool = "error_model_triphone_rich"
        for seed in ["1", "2", "3"] :
            arr_1 += triphone_rich_data[asr][dataset][seed]
        for metric in ["WER_Seed1", "WER_Seed2", "WER_Seed3"]:
            arr_2 += relative_improvement_of_finetuned_model[asr][dataset][shorten_error_model_name[tool]][metric]

assert len(arr_1) == len(arr_2)

correlation_coefficient, p_value = scipy.stats.pearsonr(arr_1, arr_2)
print()
print("Pearson's corr coeff:\t ", correlation_coefficient)
print("p-value:\t\t ", p_value)



Pearson's corr coeff:	  0.30101917837320363
p-value:		  1.6897465152817947e-10


In [29]:
## grouping by dataset

for dataset in datasets:
    arr_1 = []
    arr_2 = []
    for asr in asrs:
        tool = "error_model_triphone_rich"
        for seed in ["1", "2", "3"]:
            arr_1 += triphone_rich_data[asr][dataset][seed]
        for metric in ["WER_Seed1", "WER_Seed2", "WER_Seed3"]:
            arr_2 += relative_improvement_of_finetuned_model[asr][dataset][shorten_error_model_name[tool]][metric]

    assert len(arr_1) == len(arr_2)

    correlation_coefficient, p_value = scipy.stats.pearsonr(arr_1, arr_2)

    print()
    print("Dataset: ", dataset)
    print("Pearson's corr coeff:\t ", correlation_coefficient)
    print("p-value:\t\t ", p_value)



Dataset:  YBAA
Pearson's corr coeff:	  0.2501637338120652
p-value:		  0.14114955488410674

Dataset:  ZHAA
Pearson's corr coeff:	  0.41489133978836024
p-value:		  0.011867842980788834

Dataset:  ASI
Pearson's corr coeff:	  0.31099663771576097
p-value:		  0.06486216354917934

Dataset:  TNI
Pearson's corr coeff:	  0.02807711405430158
p-value:		  0.8708733760067474

Dataset:  NCC
Pearson's corr coeff:	  0.6257973032357591
p-value:		  4.4708141118804514e-05

Dataset:  TXHC
Pearson's corr coeff:	  0.46170134507687743
p-value:		  0.004589216953355651

Dataset:  EBVS
Pearson's corr coeff:	  0.40574036129699925
p-value:		  0.014082076068900117

Dataset:  ERMS
Pearson's corr coeff:	  0.18538357160754812
p-value:		  0.27904671661397057

Dataset:  YDCK
Pearson's corr coeff:	  0.38533844521446226
p-value:		  0.020293928658936044

Dataset:  YKWK
Pearson's corr coeff:	  0.23238271948945172
p-value:		  0.17261644774372498

Dataset:  THV
Pearson's corr coeff:	  0.4886478297241828
p-value:		  0.0024955